In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
torch.manual_seed(1)

# Discrete Probability Distribution (이산확률분포)

In [5]:
# p(주먹|가위) p(가위|가위) p(보|가위)를 예측!

# Softmax : max 값을 soft 하게
- 원래 : max(0,0,1)
- softmax : softmax(a,b,c) (a+b+c =1)

In [6]:
z = torch.FloatTensor([1,2,3])

In [7]:
hypothesis = F.softmax(z, dim=0) # -> 나중에 functional 라이브러리 문서 보기
print(hypothesis) # 확률값으로
hypothesis.sum()  

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

# Cross Entropy Loss

In [8]:
z = torch.rand(3,5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) # 2번째 dim에 대하여 소프트 맥스를 사용함
print(hypothesis)
# 이렇게 나온 값이 prediction!!

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [9]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


In [10]:
# class : 5, sample : 3
y_one_hot = torch.zeros_like(hypothesis) # hypothesis 와 같은 모양으로
y_one_hot.scatter_(1,y.unsqueeze(1),1)
# dim=1에 대해서 y를 unsqueeze한 후 1로

# unsqeeze 메서드 공부 더 해야겠다    

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [11]:
# cost
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)  

tensor(1.4689, grad_fn=<MeanBackward0>)


# 더 쉽게 하기

In [12]:
# log softmax function
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [13]:
(y_one_hot * F.log_softmax(z,dim=1)).sum(dim=1).mean()

tensor(-1.4689, grad_fn=<MeanBackward0>)

# 한 번 더 간편하게 바꾸기

In [14]:
F.nll_loss(F.log_softmax(z,dim=1), y)
# nll : negative log likelihood

tensor(1.4689, grad_fn=<NllLossBackward>)

# 진짜 더 편하게

In [15]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

# 실습 코드

### low-level cross entropy loss

In [16]:
# 데이터

x_train = [
    [1,2,1,1],
    [2,1,3,2],
    [3,1,3,4],
    [4,1,5,5],
    [1,7,5,5],
    [1,2,5,6],
    [1,6,6,6],
    [1,7,7,7]
]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [19]:
# training

# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost: {:.6f}".format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


### training with F.cross_entropy

In [21]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### High-level implementation with nn.Module

In [22]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
    def forward(self, x):
        return self.linear(x)

In [23]:
model = SoftmaxClassifierModel()

In [24]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
